In [ ]:
import sys
from pathlib import Path

current_dir = Path.cwd()
parent_dir = current_dir.parent
print(f"root_dir: {parent_dir}")
sys.path.append(str(parent_dir))

In [ ]:
from wfx.processors.airflow import AirflowProcessor
from wfx.processors.gsheet import GsheetProcessor
import json

In [ ]:
dag_name = "data_processing"
gsheet_path = parent_dir / "inputs/mappings/" / dag_name / "task_list_all.csv"
input_dag_path = parent_dir / "inputs/airflow_dags"
output_workflow_path = parent_dir / "resources/jobs"
output_workflow_name = "sample_databricks_etl"

print(f"dag_name: {dag_name}")
print(f"gsheet_path: {gsheet_path}")
print(f"input_dag_path: {input_dag_path}")
print(f"output_workflow_path: {output_workflow_path}")
print(f"output_workflow_name: {output_workflow_name}")

In [ ]:
gsheet_processor = GsheetProcessor(dag_name, str(gsheet_path))
airflow_processor = AirflowProcessor(dag_name, gsheet_processor, str(input_dag_path))
target_tasks = airflow_processor.build_save_target_tasks()

In [ ]:
gsheet_processor.get_old_task_name_list()

In [ ]:
print(json.dumps(target_tasks, indent=2))

In [ ]:
target_tasks = airflow_processor.build_target_tasks()
target_tasks

In [ ]:
type(target_tasks)

In [ ]:
from wfx.core.converter import DBXDabConverter
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

target_workflow_name = "sample_databricks_etl"
output_workflow_path = PROJECT_ROOT / "resources/jobs"

converter = DBXDabConverter()

dabs_job = converter.convert_job_config(target_tasks, target_workflow_name)
write_path = str(output_workflow_path / target_workflow_name) + ".yml"

converter.save_dabs_yaml(dabs_job, str(write_path))
print(f"Successfully created DABs job configuration: {write_path}")